In [2]:
import requests
from bs4 import BeautifulSoup
import time

# 定义基础URL和请求头
base_url = 'https://myanimelist.net/topanime.php'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# 存储所有动漫信息的列表
anime_list = []

# 页数控制（每页显示50个动漫）
page = 0

# 循环直到抓取到100个动漫为止
while len(anime_list) < 100:
    try:
        # 构建分页URL
        url = f"{base_url}?limit={page * 50}"
        
        # 发送请求
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # 检查请求是否成功
        
        # 使用BeautifulSoup解析页面
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找每个动漫条目
        anime_entries = soup.find_all('tr', {'class': 'ranking-list'})

        for entry in anime_entries:
            # 抓取标题
            title_tag = entry.find('h3', {'class': 'anime_ranking_h3'})
            title = title_tag.text.strip() if title_tag else 'Title not found'

            # 抓取评分
            score_tag = entry.find('td', {'class': 'score'})
            score = score_tag.text.strip() if score_tag else 'Score not found'

            # 抓取详情页URL
            anime_url_tag = title_tag.find('a') if title_tag else None
            anime_url = anime_url_tag['href'] if anime_url_tag else None

            # 抓取动漫详情页的简介
            synopsis = 'Synopsis not found'  # 默认值
            if anime_url:
                try:
                    # 请求动漫详情页
                    anime_response = requests.get(anime_url, headers=headers)
                    anime_response.raise_for_status()  # 检查请求是否成功
                    anime_soup = BeautifulSoup(anime_response.text, 'html.parser')
                    synopsis_tag = anime_soup.find('p', {'itemprop': 'description'})
                    synopsis = synopsis_tag.text.strip() if synopsis_tag else 'Synopsis not found'
                except requests.RequestException as e:
                    print(f"Failed to retrieve anime details from {anime_url}. Error: {e}")

            # 添加到动漫列表
            anime_list.append({
                'title': title,
                'score': score,
                'synopsis': synopsis
            })

            # 如果已经抓取到100个动漫，停止抓取
            if len(anime_list) >= 100:
                break

        # 打印当前进度
        print(f"Page {page + 1} processed. Total anime fetched: {len(anime_list)}")

        # 翻到下一页
        page += 1

        # 为了避免过于频繁的请求，添加一个延迟
        time.sleep(1)

    except requests.RequestException as e:
        print(f"Failed to retrieve page {page + 1}. Error: {e}")
        break

# 打印抓取的动漫信息
for i, anime in enumerate(anime_list, 1):
    print(f"{i}. Title: {anime['title']}, Score: {anime['score']}")
    print(f"   Synopsis: {anime['synopsis']}\n")

Page 1 processed. Total anime fetched: 50
Page 2 processed. Total anime fetched: 100
1. Title: Sousou no Frieren, Score: 9.32
   Synopsis: During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them.

However, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her "usual" routine of collecting spells across the continent. Due to her different sense of time, she seemingly holds no strong feelings toward the experiences she went through.

As the years pass, Frieren gradually realizes how her days in the hero's party truly impacted her. Witnessing the deaths of two of her former companions, Frieren begins to regret havin